In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from hansolo import *
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

Zero hidden layer

In [ ]:
N = 2000 #number of timesteps
p = 0.5 # spiking proba input neurons 
nu = 0.5 # bias

K_input = 6 #number of input neurons

K_output = 2  #number of output neurons

para_PWA = 2 #parameter expert PWA

eta_output = 0.05 #parameter EWA output layer

Nb_simu = 100 #number of simulations

expert2 = 'PWA' #choose 'EWA' for results with EWA

accuracy = np.zeros((Nb_simu))

list_obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
] #3 shapes, 3 colors

M_out = 360
M_test = 99

task = 'easy1' #choose 'easy2' for second task

if task == 'easy1':
    propA = 1/3
     #in A every obj with a 1 in 0

if task == 'easy2':
    propA = 1/2

propB = 1 - propA

nb_epoch = int(M_out / 9)
accuracy_all = np.zeros((nb_epoch, Nb_simu))


for nb in range(Nb_simu):
    L=list_obj_random(list_obj,M_out) #list of objects by blocs with random order for each bloc
#Simulation input neurons
    #L_test = list_obj_random(list_obj, M_test)
    prob_input = np.transpose(np.array([p*L for i in range(N)]), axes = (2,0,1) )
    input_neurons_out = stats.bernoulli.rvs(prob_input)
    #prob_input = np.transpose(np.array([p*L_test for i in range(N)]), axes = (2,0,1))
    #input_neurons_test = stats.bernoulli.rvs(prob_input) #to test
    P_input = np.sum(input_neurons_out, axis = 1) / N

#parameters output layer
 
    cum_gain_output = np.zeros((K_output))
    cum_gain_mid2_to_output = np.zeros((K_output, K_input))

    W_output = np.zeros((K_output, K_input, M_out+1))
    W_output_not_renorm = np.zeros((K_output, K_input, M_out+1))
    W_output[:, :, 0] += 1/K_input
    W_output_not_renorm[:, :, 0] += 1

#training output layer
    ind_epoch = 0
    for m in range(M_out):
        obj = L[m,:]
        gain_output = GainOutputEasyTask(K_output, K_input, P_input[:,m], obj, task)
        cum_gain_mid2_to_output += gain_output
        cum_gain_output += np.sum(W_output[:, :, m] * gain_output, axis = 1)
        
        if expert2 == "EWA":
            W_output_not_renorm[ :, :, m+1] = EWA(W_output_not_renorm[:, :, m], eta_output, gain_output)
            W_output[:,:, m+1] = (W_output_not_renorm[:, :, m+1].T / np.sum(W_output_not_renorm[:, :, m + 1], axis = 1)).T

        else:
            W_output_not_renorm[:, :, m+1] = PWA(para_PWA, K_output, K_input, cum_gain_output, cum_gain_mid2_to_output)
            not_zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) != 0)[0]
            zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) == 0)[0]
            W_output[not_zero,:, m + 1] = (
                W_output_not_renorm[not_zero, :, m + 1].T / np.sum(W_output_not_renorm[not_zero,:, m + 1], axis=1)
                ).T
            W_output[zero,:, m+1] = 1/K_input

#test with new images
        if m%9 == 0:
            answers_test = np.zeros((M_test))
            L_test = list_obj_random(list_obj, M_test)
            prob_input = np.transpose(np.array([p*L_test for i in range(N)]), axes = (2,0,1))
            input_neurons_test = stats.bernoulli.rvs(prob_input) #to test
            for l in range(M_test):
     
            # Simulation output neurons (without Kalikow)
                probas_output = np.sum(W_output[:,None,:,m+1] * input_neurons_test[:,:,l].T, axis = 2)
                probas_output = np.clip(probas_output, 0, 1)
                output_neurons = stats.bernoulli.rvs(probas_output)

                P_output = np.sum(output_neurons, axis = 1) / N
                chosen_class = np.where(P_output == np.max(P_output))[0]

                obj = L_test[l,:]
                if (chosen_class[0] == 0 and in_A(obj,task)) or (chosen_class[0] == 1 and in_B(obj,task)):
                    answers_test[l] = 1
            #accuracy[nb] = np.sum(answers_test)/M_test
            accuracy_all[ind_epoch, nb] = np.sum(answers_test)/M_test
            #print(nb, ind_epoch, accuracy_all[ind_epoch, nb])
            ind_epoch+=1
    print(nb)

#np.save(f"accuracy_{task}_{expert2}_0hid", accuracy_all)



One hidden layer

In [ ]:
N = 2000 #number of timesteps
p = 0.5 # spiking proba input neurons 
nu = 0.5 # bias

K_input = 6 #number of input neurons

K_output = 2  #number of output neurons

para_PWA = 2 #parameter expert PWA

eta_output = 0.05 #parameter EWA output layer
eta_mid = 3

Nb_simu = 100 #number of simulations

expert2 = 'PWA'
expert1 = 'PWA'

accuracy = np.zeros((Nb_simu))

list_obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
] #3 shapes, 3 colors

M_out = 360
M_hid = 90
M_sel = 18
M_test = 99

task = 'easy2' #choose 'easy1' for first task

if task == 'easy1':
    propA = 1/3
     #in A every obj with a 1 in 0

if task == 'easy2':
    propA = 1/2

propB = 1 - propA

K_mid = int(K_input * (K_input - 1) /2)
#K_mid_new = 9

thresh = 0.1 #threshold to select neurons

indices = np.arange(K_input)
i, j = np.meshgrid(indices, indices)
mask = i < j
l_mid = np.column_stack((i[mask], j[mask]))

nb_epoch = int(M_out / 9)

accuracy_all = np.zeros((nb_epoch, Nb_simu))

for nb in range(Nb_simu):
    #print(nb)
    L_out=list_obj_random(list_obj,M_out) #list of objects by blocs with random order for each bloc
    L_hid = list_obj_random(list_obj,M_hid)
    L_sel = list_obj_random(list_obj,M_sel)

    #Simulation input neurons
    #to train mid neurons
    prob =  np.transpose(np.array([p*L_hid for i in range(N)]), axes = (2,0,1) )
    input_neurons_mid = stats.bernoulli.rvs(prob) #to train mid neurons

    #to sel mid neurons
    prob =  np.transpose(np.array([p*L_sel for i in range(N)]), axes = (2,0,1) )
    input_neurons_sel = stats.bernoulli.rvs(prob) 

    #to train output neurons
    prob_input = np.transpose(np.array([p*L_out for i in range(N)]), axes = (2,0,1) )
    input_neurons_out = stats.bernoulli.rvs(prob_input)


    #Parameters mid neurons
    W_mid = np.zeros((K_mid,K_input,M_hid+1))
    W_mid[:,:,0] = 1/K_input
    W_mid_not_renorm = np.zeros((K_mid, K_input, M_hid+1))
    W_mid_not_renorm[:, :, 0] += 1

    cum_gain_mid = np.zeros((K_mid))
    cum_gain_input_to_mid = np.zeros((K_mid,K_input))

#training mid neurons

    for m in range(M_hid):
        
        gain_mid = GainMid(np.arange(K_mid),K_mid,K_input,input_neurons_mid[:,:,m], l_mid) #(K_mid,K_input)
        cum_gain_input_to_mid += gain_mid
        cum_gain_mid += np.sum(W_mid[:,:,m] * gain_mid, axis = 1)

        if expert1 == "EWA":
            W_mid_not_renorm[:, :, m + 1] = EWA(W_mid_not_renorm[:, :, m], eta_mid, gain_mid)
            W_mid[:, :, m + 1] = (W_mid_not_renorm[:, :, m + 1].T / np.sum(W_mid_not_renorm[:, :, m + 1], axis=1)).T
            #W_mid[:, :, m+1] = 1/K_mid
        else:
            W_mid_not_renorm[:, :, m + 1] = PWA(para_PWA, K_mid, K_input, cum_gain_mid, cum_gain_input_to_mid) 
            not_zero = np.where(np.sum(W_mid_not_renorm[:,:,m+1], axis = 1) != 0)[0]
            zero = np.where(np.sum(W_mid_not_renorm[:,:,m+1], axis = 1) == 0)[0]
            W_mid[not_zero, :, m + 1] = (W_mid_not_renorm[not_zero, :, m + 1].T / np.sum(W_mid_not_renorm[not_zero, :, m + 1], axis=1)).T
            W_mid[zero, :, m+1] = 1/K_input
                
    #selection mid neurons

    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[:,:,M_hid], input_neurons_sel)
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_sel = stats.bernoulli.rvs(clipped_probas_mid)
    P_mid = np.sum(mid_neurons_sel, axis=1) / N
    max_P = np.max(P_mid, axis = 1 )
    max_P_sorted = np.sort(max_P)

    #x = max_P_sorted[K_mid-K_mid_new] 
    x = thresh
    mid_neur_ok = np.where(max_P >= x)[0] #selected neurons
    K_mid_new = np.shape(mid_neur_ok)[0]

    mid_neur = mid_neur_ok
    mid_neur = np.sort(mid_neur)

    prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[mid_neur,:,M_hid], input_neurons_out)
    clipped_probas_mid = np.clip(prob_mid, 0, 1)
    mid_neurons_out=stats.bernoulli.rvs(clipped_probas_mid)

    P_mid = np.sum(mid_neurons_out, axis = 1) / N

    #parameters output
    
    cum_gain_output = np.zeros((K_output))
    cum_gain_mid2_to_output = np.zeros((K_output, K_mid_new))

    W_output = np.zeros((K_output, K_mid_new, M_out+1))
    W_output_not_renorm = np.zeros((K_output, K_mid_new, M_out+1))
    W_output[:, :, 0] += 1/K_mid_new
    W_output_not_renorm[:, :, 0] += 1

    ind_epoch = 0
    for m in range(M_out):
        obj = L_out[m,:]
        gain_output = GainOutputEasyTask(K_output, K_mid_new, P_mid[:,m], obj, task) 
        cum_gain_mid2_to_output += gain_output
        cum_gain_output += np.sum(W_output[:, :, m] * gain_output, axis = 1)
            

        if expert2 == "EWA":
            W_output_not_renorm[ :, :, m+1] = EWA(W_output_not_renorm[:, :, m], eta_output, gain_output)
            W_output[:,:, m+1] = (W_output_not_renorm[:, :, m+1].T / np.sum(W_output_not_renorm[:, :, m + 1], axis = 1)).T
        else:
            W_output_not_renorm[:, :, m+1] = PWA(para_PWA, K_output, K_mid_new, cum_gain_output, cum_gain_mid2_to_output)
                
            not_zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) != 0)[0]
            zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) == 0)[0]
            W_output[not_zero,:, m + 1] = (
                W_output_not_renorm[not_zero, :, m + 1].T / np.sum(W_output_not_renorm[not_zero,:, m + 1], axis=1)
                ).T
            W_output[zero,:, m+1] = 1/K_mid_new
            
        
        if m%9 == 0:
            #to test 
            L_test = list_obj_random(list_obj,M_test)
            prob_input = np.transpose(np.array([p*L_test for i in range(N)]), axes = (2,0,1) )
            input_neurons_test = stats.bernoulli.rvs(prob_input) 


            #test with new images
            prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[mid_neur,:,-1], input_neurons_test)
            clipped_probas_mid = np.clip(prob_mid, 0, 1)
            mid_neurons_test=stats.bernoulli.rvs(clipped_probas_mid)

            answers_test = np.zeros((M_test))
            for l in range(M_test):
     
            # Simulation output neurons (without Kalikow)
                probas_output = np.sum(W_output[:,None,:,m+1] * mid_neurons_test[:,:,l].T, axis = 2)
                probas_output = np.clip(probas_output, 0, 1)
                output_neurons = stats.bernoulli.rvs(probas_output)

                P_output = np.sum(output_neurons, axis = 1) / N
                chosen_class = np.where(P_output == np.max(P_output))[0]

                obj = L_test[l,:]
                if (chosen_class[0] == 0 and in_A(obj,task)) or (chosen_class[0] == 1 and in_B(obj,task)):
                    answers_test[l] = 1
            accuracy_all[ind_epoch, nb] = np.sum(answers_test)/M_test
            ind_epoch+=1
        
    print(nb)

#np.save(f"accuracy_{task}_{expert2}_1hid", accuracy_all)

